http://web.mta.info/developers/turnstile.html
https://groups.google.com/forum/#!searchin/mtadeveloperresources/turnstile/mtadeveloperresources/tynsUKSJtjk/BIdNU_kdKnwJ

C/A      = Control Area (A002)
UNIT     = Remote Unit for a station (R051)
SCP      = Subunit Channel Position represents an specific address for a device
           (02-00-00)
STATION  = Represents the station name the device is located at
LINENAME = Represents all train lines that can be boarded at this station
           Normally lines are represented by one character.
           LINENAME 456NQR repersents train server for
           4, 5, 6, N, Q, and R trains.

DIVISION = Represents the Line originally the station belonged to BMT, IRT,
           or IND
DATE     = Represents the date (MM-DD-YY)
TIME     = Represents the time (hh:mm:ss) for a scheduled audit event
DESc     = Represent the "REGULAR" scheduled audit event
           (Normally occurs every 4 hours)
           1. Audits may occur more that 4 hours due to planning,
           or troubleshooting activities.
           2. Additionally, there may be a "RECOVR AUD" entry:
           This refers to a missed audit that was recovered.

ENTRIES  = The cumulative entry register value for a device
EXIST    = The cumulative exit register value for a device

In [18]:
import pandas as pd
import dateutil.parser

def date_year_week(date_str):
    parsed_date = dateutil.parser.parse(date_str).isocalendar()
    return '{}-{}'.format(parsed_date[0], parsed_date[1])
    
def calculate_entry(list_date_entry):
    """
    each item in `list_date_entry` is another list [date, entry]
    """
    list_date_entry.sort(key=lambda x: x[1])
    lowest_entry = list_date_entry[0][1]
    highest_entry = list_date_entry[-1][1]
    #print(list_date_entry)
    return highest_entry - lowest_entry

# Calculate counts for each turnstile per week

columns = ['control_area', 'unit', 'scp', 'station', 'line_name',
            'division', 'date', 'time', 'desc', 'entries', 'exits']
df = pd.read_csv(
        #'turnstile_170624.txt'
       'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170624.txt'
)
df.columns = columns

#df = df[(df.station == '59 ST')
#        & ((df.date == '06/17/2017') | (df.date == '06/18/2017'))].sort_values(['time', 'scp'])

df['value_date_entry'] = df.apply(lambda row: [dateutil.parser.parse(row.date  + ' ' + row.time), row.entries], axis=1)
df['index_week'] = df.apply(lambda row: (row.control_area, row.unit, row.scp, row.station, date_year_week(row.date)), axis=1)

df_group_week = df.groupby('index_week')['value_date_entry'].apply(list)

# Currently a series, convert to a dataframe
df_group_week = pd.DataFrame({'date_and_entries': df_group_week})
df_group_week['entries_total'] = df_group_week['date_and_entries'].map(calculate_entry)
df_group_week.sample(30)

,date_and_entries,entries_total
index_week,,
"(B019, R149, 00-00-04, NEWKIRK PLAZA, 2017-24)","[[2017-06-17 01:00:00, 14890279], [2017-06-17 ...",2732
"(R169, R168, 01-03-00, 96 ST, 2017-25)","[[2017-06-19 01:00:00, 3066349], [2017-06-19 0...",5446
"(TRAM1, R468, 00-00-02, RIT-MANHATTAN, 2017-25)","[[2017-06-19 01:00:00, 1516269], [2017-06-19 0...",3677
"(PTH22, R540, 02-00-01, PATH NEW WTC, 2017-25)","[[2017-06-19 00:58:50, 30900], [2017-06-19 05:...",3266
"(R550, R072, 00-05-01, 34 ST-HUDSON YD, 2017-24)","[[2017-06-17 00:00:00, 1], [2017-06-17 04:00:0...",0
"(R129, R321, 00-00-02, 18 ST, 2017-24)","[[2017-06-17 00:00:00, 13715444], [2017-06-17 ...",1751
"(R422, R428, 00-05-00, BUHRE AV, 2017-24)","[[2017-06-17 01:00:00, 317], [2017-06-17 05:00...",0
"(R159, R164, 01-00-02, 66 ST-LINCOLN, 2017-24)","[[2017-06-17 01:00:00, 5091516], [2017-06-17 0...",1167
"(C022, R212, 01-05-01, 59 ST, 2017-24)","[[2017-06-17 00:00:00, 0], [2017-06-17 04:00:0...",0
